# Person re-identification 

### This sample use simple pretrained resnet-152 model to extract features/embeddings for an image 

### Compute Embedding

In [1]:
images_dir = "/tmp/Market-1501-v15.09.15/bounding_box_train/"

In [2]:
import sys
import os
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import torch
import pandas as pd
sys.path.append(os.path.abspath('./src'))
from resnetembedding import ResnetEmbedder
from datasets.Market1501Dataset import Market1501Dataset


In [5]:
def embed(image_rgb):
    embedder = ResnetEmbedder()
    return embedder(image_rgb).data.numpy()


In [ ]:
%%time 

images_dataset = Market1501Dataset(images_dir)

embeddings = []
class_person=[]
for person_img, target in images_dataset:
    embeddings.append(  embed(person_img)[0])
    class_person.append(target)

embeddings = np.stack( embeddings)


In [ ]:
embeddings[0:5]

In [ ]:
embeddings.shape

### Compute pairwise distance

In [ ]:
from EuclideanPairwiseDistance import EuclideanPairwiseDistance

distance_metric = EuclideanPairwiseDistance()
pairwise_distance = distance_metric(torch.tensor(embeddings))
pairwise_distance

### Compute CMC score
Definition: https://www.nist.gov/sites/default/files/documents/2016/12/06/12_ross_cmc-roc_ibpc2016.pdf

* Each sample is  compared against all gallery samples. The resulting scores are sorted and ranked

* Determine the rank at which a true match occurs 
* True Positive Identification Rate (TPIR): Probability of observing the correct identity within the top K ranks
* CMC Curve: Plots TPIR against ranks
* CMC Curve: Rank based metric



In [ ]:
from EuclideanPairwiseDistance import EuclideanPairwiseDistance

distance_metric = EuclideanPairwiseDistance()
pairwise_distance = distance_metric(torch.tensor(embeddings))
pairwise_distance


In [ ]:
from CMCScore import CMCScore

cmc_scorer = CMCScore()
score = cmc_scorer.score(pairwise_distance, class_person, 2)
print(score)

In [ ]:
k=1
accuracy, rank_k, rank_k_label = cmc_scorer.accuracy_at_top_k(pairwise_distance,  class_person, k)
print("Percentage accuracy at k = {} is {}".format(k, accuracy ))

In [ ]:
rank_k

In [ ]:
result_df = pd.DataFrame(rank_k_label, columns= ["predicted_k_{}".format(i) for i in range(1, k+1)])
result_df["actual"] = class_person

In [ ]:
result_df.head(n=20)

### Visualise the embeddings extracted using T-SNE

In [ ]:
!pip install scikit-learn==0.21.2

In [ ]:
from sklearn.manifold import TSNE
embedding_tsne = TSNE(random_state=42).fit_transform(embeddings)

print("embeddings reduced to 2-D using TSNE\n")
print(embedding_tsne[0:10])

In [ ]:
group = list(set(class_person))
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

scatter = ax.scatter(embedding_tsne[:, 0], embedding_tsne[:, 1], c= class_person, alpha=0.5, s= 200, cmap="Spectral" )

plt.title('T-SNE plot of person embedding')
plt.show()